1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때 , 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [3]:
%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
docs = loader.load_and_split(text_splitter)

[Document(metadata={'source': './tax.docx'}, page_content='소득세법\n\n[시행 2025. 1. 1.] [법률 제20247호, 2024. 12. 31., 일부개정]\n\n제1장 총칙\n\n제1조(목적)\n\n이 법은 개인의 소득에 대하여 소득세를 부과·징수함으로써 국가의 재정수입을 확보하고 소득의 재분배를 도모함을 목적으로 한다.\n\n제2조(정의)\n\n이 법에서 사용하는 용어의 뜻은 다음과 같다.\n\n"거주자"란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n"비거주자"란 거주자가 아닌 개인을 말한다.\n\n"소득"이란 해당 과세기간에 발생한 다음 각 목의 소득을 말한다.\n\n이자소득\n\n배당소득\n\n사업소득\n\n근로소득\n\n연금소득\n\n기타소득\n\n퇴직소득\n\n양도소득\n\n제3조(과세소득의 범위)\n\n① 거주자에 대해서는 이 법에 따라 모든 소득에 대하여 과세한다.\n\n② 비거주자에 대해서는 국내원천소득에 대해서만 과세한다.\n\n제4조(과세기간)\n\n① 소득세의 과세기간은 1월 1일부터 12월 31일까지 1년으로 한다.\n\n② 거주자가 사망한 경우의 과세기간은 1월 1일부터 사망한 날까지로 한다.\n\n제2장 이자소득\n\n제16조(이자소득)\n\n이자소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다.\n\n국가 또는 지방자치단체가 발행한 채권 또는 증권의 이자와 할인액\n\n내국법인이 발행한 채권 또는 증권의 이자와 할인액\n\n국내에서 받는 예금의 이자\n\n채권 또는 증권의 환매조건부 매매차익\n\n저축성보험의 보험차익\n\n비영업대금의 이익\n\n그 밖에 대통령령으로 정하는 소득\n\n제3장 배당소득\n\n제17조(배당소득)\n\n배당소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다.\n\n내국법인으로부터 받는 이익이나 잉여금의 배당 또는 분배금\n\n법인으로 보는 단체로부터 받는 배당금 또는 분배금\n\n의제배당(「법인세법」에

In [29]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [37]:
from langchain_chroma import Chroma

# db = Chroma.from_documents(
#     docs,
#     embeddings,
#     collection_name="chroma-tax",
#     persist_directory="./chroma",
# )

db = Chroma(
    collection_name="chroma-tax",
    persist_directory="./chroma",
    embedding_function=embeddings
)

In [38]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# retrieved_docs = db.similarity_search(query, k=3)

In [39]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [ ]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [40]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

In [41]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [53]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [54]:
ai_message = qa_chain.invoke(query)

In [55]:
ai_message

'정확한 금액은 알 수 없습니다. 연봉 5,000만원에 대한 실효 소득세는 근로소득공제·기본공제(가족 수)·건강보험·국민연금 등 각종 소득공제 및 세액공제 반영 후 과세표준에 세율을 적용해 계산해야 하므로 가족수, 공제 항목 금액, 기타소득 여부 등의 정보가 필요합니다. 필요한 항목을 알려주시면 대략 계산해 드리겠습니다.'